<a href="https://colab.research.google.com/github/raikanwalrai/PowerBI/blob/main/009.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
#pip install pandas openpyxl networkx pyvis

In [14]:

from google.colab import drive

MyDrive = '/00_00_COLA'
#os.chdir(MyDrive)

# Mount Google Drive
#drive.mount('/00_00_COLA')
drive.mount(MyDrive)



Drive already mounted at /00_00_COLA; to attempt to forcibly remount, call drive.mount("/00_00_COLA", force_remount=True).


In [20]:
import os
MyDrive = '/00_00_COLA/MyDrive/00_00_COLA'
os.chdir(MyDrive)
os.listdir(MyDrive)

['superstore_sales.xlsx',
 'Untitled0.ipynb',
 'Copy of Multiclass-BERT.ipynb',
 'cluster_mpg.csv',
 'Untitled1.ipynb',
 'Copy of Sentiment.csv',
 '01_Market_Basket_Optimisation_00.csv',
 '01_Market_Basket_Optimisation_01.csv',
 'apriori+based+recommendation.ipynb',
 '00-Decision-Trees.ipynb',
 'PYTHON_BOOK',
 'Copy of image_1.png',
 'US20230119590A1.pdf',
 'MATHS_AQ4.pdf',
 'Conjont.xlsx',
 'conjointdata.csv',
 'Citations_1_50.xlsx',
 'file1.xls',
 'Superstore.xlsx',
 'sales_trend.png',
 'DAY1.xlsx']

In [29]:
import pandas as pd
import networkx as nx
from pyvis.network import Network

# ---------- Read Excel ----------
df = pd.read_excel('DAY1.xlsx')

In [30]:
# ---------- Create HTML Table ----------
table_html = df.to_html(
    index=False,
    classes="schedule-table",
    border=0,
    justify="center"
)

In [31]:
# ---------- Build PERT / Flow Graph ----------
G = nx.DiGraph()

for i, row in df.iterrows():
    label = f"{row['Task']}\n{row['Start Time']} - {row['End Time']}\n{row['Duration']} mins"
    G.add_node(row['Task'], label=label)

    if i > 0:
        G.add_edge(df.iloc[i - 1]['Task'], row['Task'])

In [32]:
# ---------- Convert to Interactive Network ----------
net = Network(
    height="400px",
    width="100%",
    directed=True,
    bgcolor="#ffffff",
    font_color="black"
)

net.from_nx(G)
net.save_graph("pert.html")

In [28]:
# ---------- Final Combined HTML ----------
final_html = f"""
<html>
<head>
<style>
body {{
    font-family: Arial, sans-serif;
    margin: 40px;
}}
.schedule-table {{
    width: 100%;
    border-collapse: collapse;
    margin-bottom: 40px;
}}
.schedule-table th {{
    background: #2c3e50;
    color: white;
    padding: 10px;
}}
.schedule-table td {{
    border: 1px solid #ccc;
    padding: 8px;
}}
.schedule-table tr:nth-child(even) {{
    background: #f9f9f9;
}}
</style>
</head>

<body>

<h2>Course Schedule</h2>
{table_html}

<h2>Session Flow (PERT View)</h2>
<iframe src="pert.html" width="100%" height="450" frameborder="0"></iframe>

</body>
</html>
"""

with open("schedule_dashboard.html", "w", encoding="utf-8") as f:
    f.write(final_html)

print("✅ HTML schedule_dashboard.html created")

✅ HTML schedule_dashboard.html created


In [35]:
!pip install streamlit pandas networkx pyvis openpyxl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 59.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 96.8 MB/s eta 0:00:00


In [42]:
import streamlit as st
import pandas as pd
import networkx as nx
from pyvis.network import Network
import tempfile
import os

In [43]:
# -------------------------------
# Page Config (Deloitte style)
# -------------------------------
st.set_page_config(
    page_title="Training Delivery Dashboard",
    layout="wide"
)

2026-01-30 08:25:13.095 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


In [44]:
st.markdown("## 🎓 Training Delivery Dashboard")
st.markdown("**Enterprise Architecture | Professional Learning | Delivery View**")

2026-01-30 08:25:28.010 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-01-30 08:25:28.011 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-01-30 08:25:28.012 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-01-30 08:25:28.013 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-01-30 08:25:28.013 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-01-30 08:25:28.014 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


DeltaGenerator()

In [48]:
# -------------------------------
# Upload Excel
# -------------------------------
uploaded_file = st.file_uploader(
    "DAY1",
    type=["xlsx"]
)

if uploaded_file:
    schedule = pd.read_excel(uploaded_file, sheet_name="Schedule")
    deps = pd.read_excel(uploaded_file, sheet_name="Dependencies")

    # -------------------------------
    # KPIs
    # -------------------------------
    total_duration = schedule["Duration"].sum()
    session_count = schedule[schedule["Type"] == "Session"].shape[0]
    break_count = schedule[schedule["Type"] == "Break"].shape[0]

    col1, col2, col3 = st.columns(3)
    col1.metric("Total Duration (mins)", total_duration)
    col2.metric("Sessions", session_count)
    col3.metric("Breaks", break_count)

    # -------------------------------
    # Schedule Table
    # -------------------------------
    st.markdown("### 📋 Session Schedule")

    def highlight_breaks(row):
        return [
            "background-color: #f2f2f2" if row.Type == "Break" else ""
        ] * len(row)

    st.dataframe(
        schedule.style.apply(highlight_breaks, axis=1),
        use_container_width=True
    )

    # -------------------------------
    # Build PERT Network
    # -------------------------------
    G = nx.DiGraph()

    for _, row in schedule.iterrows():
        label = (
            f"{row['Task']}\n"
            f"{row['Start']} - {row['End']}\n"
            f"{row['Duration']} mins"
        )

        color = "#1f77b4" if row["Type"] == "Session" else "#b0b0b0"

        G.add_node(
            row["ID"],
            label=label,
            title=row["Description"],
            color=color
        )

    for _, d in deps.iterrows():
        G.add_edge(d["From"], d["To"])

    # -------------------------------
    # Render Interactive Graph
    # -------------------------------
    net = Network(
        height="500px",
        width="100%",
        directed=True,
        bgcolor="#ffffff"
    )

    net.from_nx(G)
    net.toggle_physics(True)

    tmp_file = tempfile.NamedTemporaryFile(delete=False, suffix=".html")
    net.save_graph(tmp_file.name)


    print(tmp_file.name)

    st.markdown("### 🔗 Learning Flow (PERT / ADM View)")
    st.components.v1.html(
        open(tmp_file.name, "r", encoding="utf-8").read(),
        height=550
    )

    # -------------------------------
    # Executive Notes
    # -------------------------------
    st.markdown("### 🧠 Delivery Notes")
    st.markdown("""
    - Sessions are sequenced using **PERT-style dependencies**
    - Breaks are modeled as **control nodes**
    - Supports **ADM-style phased delivery**
    - Can be extended to **Critical Path / Risk Analysis**
    """)

else:
    st.info("👆 Upload an Excel file to begin")


2026-01-30 08:28:27.739 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-01-30 08:28:27.741 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-01-30 08:28:27.742 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-01-30 08:28:27.743 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-01-30 08:28:27.745 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-01-30 08:28:27.746 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-01-30 08:28:27.748 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-01-30 08:28:27.750 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar